<a href="https://colab.research.google.com/github/KiranVarghese25/Project/blob/main/LLM_project_2348525.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import subprocess

# List of packages to install
packages = [
    'langchain',
    'google-search-results',
    'sentence_transformers',
    'faiss-cpu',
    'langchain-community'
]

# Install packages
for package in packages:
    subprocess.check_call(["pip", "install", package])


In [ ]:
!pip install --upgrade pydantic langchain transformers sentence-transformers faiss-cpu


  Using cached langchain-0.2.14-py3-none-any.whl.metadata (7.1 kB)
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached faiss_cpu-1.8.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.7 kB)
  Using cached langchain_core-0.2.35-py3-none-any.whl.metadata (6.2 kB)
  Using cached langsmith-0.1.104-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached langchain-0.2.14-py3-none-any.whl (997 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
Using cached faiss_cpu-1.8.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (27.0 MB)
Using cached langchain_core-0.2.35-py3-none-any.whl (394 kB)
Using cached langsmith-0.1.104-py3-none-any.whl (149 kB)
Using cached tokeniz

In [ ]:
from google.colab import files
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import os

# Upload the CSV file
uploaded = files.upload()

# Check the uploaded files
for file_name in uploaded.keys():
    print(f"Uploaded file: {file_name}")

# Load the CSV file into a DataFrame, specifying the encoding
df = pd.read_csv(next(iter(uploaded.keys())), encoding='latin-1')  # Try 'latin-1' or 'Windows-1252'
print(df.head())

# Initialize SentenceTransformer for embeddings
model_name = "hkunlp/instructor-large"
sentence_model = SentenceTransformer(model_name)
vectordb_file_path = "faiss_index"

def create_vector_db():
    # Load data from DataFrame
    documents = df['prompt'].tolist()  # Extract 'prompt' column as list

    # Compute embeddings for documents
    embeddings = sentence_model.encode(documents, convert_to_numpy=True)

    # Create a FAISS index
    dimension = embeddings.shape[1]
    nlist = min(100, len(documents))  # Number of clusters should be at most the number of documents
    quantizer = faiss.IndexFlatL2(dimension)  # Flat quantizer
    index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)

    # Train the index
    index.train(embeddings)

    # Add embeddings to the index
    index.add(embeddings)

    # Save FAISS index locally
    faiss.write_index(index, vectordb_file_path)

def get_qa_chain():
    # Load the FAISS index from the local folder
    index = faiss.read_index(vectordb_file_path)

    # Define a function to query the FAISS index
    def query_faiss(query, k=5):
        # Compute query embedding
        query_embedding = sentence_model.encode([query], convert_to_numpy=True)
        # Search for nearest neighbors
        distances, indices = index.search(query_embedding, k)
        return indices, distances

    return query_faiss

def get_responses(indices):
    # Map indices to responses
    responses = [df.iloc[i]['prompt'] for i in indices[0]]
    return responses

if __name__ == "__main__":
    create_vector_db()
    query_function = get_qa_chain()
    query = "Do you have javascript course?"
    indices, distances = query_function(query)
    responses = get_responses(indices)

    # Print responses
    print(f"Query: {query}")
    print("Top responses:")
    for response in responses:
        print(response)


Saving codebasics_faqs.csv to codebasics_faqs (6).csv
Uploaded file: codebasics_faqs (6).csv
                                              prompt  \
0  I have never done programming in my life. Can ...   
1                     Why should I trust Codebasics?   
2  Is there any prerequisite for taking this boot...   
3  What datasets are used in this bootcamp? Is it...   
4  Im not sure if this bootcamp is good enough f...   

                                            response  
0  Yes, this is the perfect bootcamp for anyone w...  
1  Till now 9000 + learners have benefitted from ...  
2  Our bootcamp is specifically designed for begi...  
3  The datasets used in this bootcamp are crafted...  
4  We got you covered. Go ahead and watch our you...  


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no lon

Query: Do you have javascript course?
Top responses:
I have never done programming and belong to a non-technical background. Can I take this course?
How do I enable Power Pivot before using it for the first time ?
How do I enable Power Pivot before using it for the first time ?
How do I enable Power Pivot before using it for the first time ?
How do I enable Power Pivot before using it for the first time ?
